In [1]:
import numpy as np
from tensorflow import keras
from sklearn import preprocessing

train_labels=np.load("dataset_nRC_train_labels.npy")
test_labels=np.load("dataset_nRC_test_labels.npy")

num_classes = len(np.unique(train_labels))
print(num_classes)


le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_labels_num = le.transform(train_labels)
test_labels_num = le.transform(test_labels)

train_labels_bin = keras.utils.to_categorical(train_labels_num, num_classes)
test_labels_bin = keras.utils.to_categorical(test_labels_num, num_classes)


13


In [3]:
from ExpConfiguration import *
from modelUtils import *
import numpy as np
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import f1_score
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

print(tf.__version__)


batch_size = 32
epochs = 10

v_acc = {}
p ='new'
nl=3
venc = {}
for en in seqEncoders:
    print(en['filename'])
    train_seq=np.load('dataset_nRC_train_' + en['filename'] + '_'+p+'_seq.npy')
    test_seq=np.load('dataset_nRC_test_' + en['filename'] + '_'+p+'_seq.npy')

    train_seq = keras.utils.to_categorical(train_seq)
    test_seq = keras.utils.to_categorical(test_seq)

    tf.keras.backend.clear_session()
    print(train_seq.shape)
    print(test_seq.shape)
    if (en['filename'] in ['1mer','2mer','3mer']):
        m=buildCNNModel2(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=1)
    else:
        m=buildCNNModel2(inshape=train_seq.shape[1:],num_classes=num_classes,nlayers=nl,cnndim=2)
    m.compile(optimizer=keras.optimizers.Adam(lr=.0005),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    m.summary()
    m.fit(train_seq, train_labels_bin,verbose=1,
                  batch_size=batch_size,shuffle=True,
                  epochs=epochs,validation_split=0.1)

            
    pred = m.predict(test_seq, verbose=1)
    predicted = np.argmax(pred, axis=1)
    venc.update({en['filename'] : {'0' : predicted}})
v_acc.update({str(nl) : venc})
print(v_acc)





1.13.1
3mer
(6160, 257, 65)
(2529, 257, 65)
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 257, 64)           12544     
_________________________________________________________________
batch_normalization_v1 (Batc (None, 257, 64)           256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 128, 64)           0         
_________________________________________________________________
gaussian_noise (GaussianNois (None, 128, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 128, 64)           0         
___________________________________________

Epoch 1/10
5544/5544 [==============================] - 7s 1ms/sample - loss: 2.4861 - acc: 0.4794 - val_loss: 3.6213 - val_acc: 0.2614
Epoch 2/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.7953 - acc: 0.6778 - val_loss: 3.3898 - val_acc: 0.2760
Epoch 3/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.5560 - acc: 0.7316 - val_loss: 3.2225 - val_acc: 0.3052
Epoch 4/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.3606 - acc: 0.7742 - val_loss: 3.2804 - val_acc: 0.3068
Epoch 5/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.2118 - acc: 0.8084 - val_loss: 3.3471 - val_acc: 0.3312
Epoch 6/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.0946 - acc: 0.8360 - val_loss: 3.5248 - val_acc: 0.3247
Epoch 7/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.0008 - acc: 0.8571 - val_loss: 3.5402 - val_acc: 0.3198
Epoch 8/10
5544/5544 [==========================

Epoch 1/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 2.8003 - acc: 0.3525 - val_loss: 5.3693 - val_acc: 0.0000e+00
Epoch 2/10
5544/5544 [==============================] - 5s 893us/sample - loss: 2.1612 - acc: 0.5406 - val_loss: 5.2238 - val_acc: 0.0990
Epoch 3/10
5544/5544 [==============================] - 5s 891us/sample - loss: 1.8329 - acc: 0.6310 - val_loss: 4.3374 - val_acc: 0.1640
Epoch 4/10
5544/5544 [==============================] - 5s 885us/sample - loss: 1.5959 - acc: 0.6962 - val_loss: 4.1909 - val_acc: 0.1721
Epoch 5/10
5544/5544 [==============================] - 5s 889us/sample - loss: 1.4143 - acc: 0.7442 - val_loss: 3.5413 - val_acc: 0.2597
Epoch 6/10
5544/5544 [==============================] - 5s 900us/sample - loss: 1.3065 - acc: 0.7700 - val_loss: 3.5702 - val_acc: 0.3052
Epoch 7/10
5544/5544 [==============================] - 5s 887us/sample - loss: 1.1840 - acc: 0.8074 - val_loss: 3.8111 - val_acc: 0.2711
Epoch 8/10
5544/5544 [==========

Epoch 1/10
5544/5544 [==============================] - 7s 1ms/sample - loss: 2.6886 - acc: 0.3819 - val_loss: 5.0418 - val_acc: 0.0000e+00
Epoch 2/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 2.1144 - acc: 0.5559 - val_loss: 4.4408 - val_acc: 0.0552
Epoch 3/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.8039 - acc: 0.6367 - val_loss: 3.2158 - val_acc: 0.2792
Epoch 4/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.5718 - acc: 0.6959 - val_loss: 3.5213 - val_acc: 0.2581
Epoch 5/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.4271 - acc: 0.7300 - val_loss: 3.7431 - val_acc: 0.2175
Epoch 6/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.2871 - acc: 0.7742 - val_loss: 3.6818 - val_acc: 0.2143
Epoch 7/10
5544/5544 [==============================] - 6s 1ms/sample - loss: 1.1760 - acc: 0.7996 - val_loss: 3.7344 - val_acc: 0.2500
Epoch 8/10
5544/5544 [======================

In [4]:
import pickle

f = open('store_acc_datasetnRC.pckl', 'wb')
pickle.dump(v_acc, f)
f.close()

f = open('store_acc_datasetnRC.pckl', 'rb')
v_acc = pickle.load(f)
f.close()



In [5]:
v_acc


{'3': {'3mer': {'0': array([0, 0, 0, ..., 3, 7, 3])},
  '2mer': {'0': array([0, 0, 0, ..., 3, 8, 3])},
  '1mer': {'0': array([0, 0, 0, ..., 3, 8, 3])},
  'Snake': {'0': array([0, 0, 0, ..., 3, 9, 3])},
  'Morton': {'0': array([0, 0, 0, ..., 3, 3, 3])},
  'Hilbert': {'0': array([0, 0, 0, ..., 3, 3, 3])}}}

In [ ]:
y_pred

In [7]:
from sklearn.metrics import *
import pandas as pd
import numpy as np
from ExpConfiguration import *


y_true = np.argmax(test_labels_bin, axis=1)
y_true = le.inverse_transform(y_true)


dfs = {}
for en in seqEncoders:
    y_pred = v_acc[str(nl)][en['filename']]['0']
    y_pred = le.inverse_transform(y_pred)
    acc=accuracy_score(y_true,y_pred)
    mcc=matthews_corrcoef(y_true,y_pred)
    p=precision_score(y_true,y_pred,average='macro')
    r=recall_score(y_true,y_pred,average='macro')
    f1=f1_score(y_true,y_pred,average='macro')
    print("%s,%.2f & %.2f & %.2f & %.2f & %.2f \\\\" % (en['filename'],acc,r,p,f1,mcc))




3mer,0.81 & 0.81 & 0.82 & 0.81 & 0.79 \\
2mer,0.83 & 0.83 & 0.84 & 0.83 & 0.82 \\
1mer,0.88 & 0.88 & 0.89 & 0.88 & 0.87 \\
Snake,0.82 & 0.82 & 0.83 & 0.81 & 0.80 \\
Morton,0.78 & 0.78 & 0.79 & 0.78 & 0.77 \\
Hilbert,0.81 & 0.81 & 0.84 & 0.82 & 0.80 \\


In [ ]:


df

In [ ]:

train_labels_bin



In [ ]:

seqEncoders[4:6]
